In [1]:
import pandas as pd
import numpy as np
import mlflow
import boto3
import os
import json
from mlflow.deployments import get_deploy_client

In [10]:
from mlflow.deployments import get_deploy_client
client = get_deploy_client("sagemaker")

In [11]:
df = pd.read_csv("./data/sp500_predict_2023-11-15.csv")
df.set_index('Date', inplace=True)
#df.drop("Date", axis=1, inplace=True)
df

,Close,Volume,Rendite,SMA 10,EMA 10,EMA 20,WMA 10,Momentum 10,SAR,RSI,ROC,%R,OBV,MACD,MACD_SIGNAL,MACD_HIST,CCI,ADOSC,%K,%D
Date,,,,,,,,,,,,,,,,,,,,
2023-11-15,2.924371,1.275609,0.105997,2.835659,2.836735,2.802099,2.857765,4.766703,2.753382,1.229881,1.824033,1.133682,2.132107,2.04021,0.477527,4.952725,0.992047,1.40634,-0.784919,0.019175


## Load Credentials

In [ ]:
from dotenv import load_dotenv
import boto3
load_dotenv()
boto3.setup_default_session(
    aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
    region_name=os.getenv('AWS_DEFAULT_REGION')
)

## Setup Deployment

In [4]:
model_uri = ".../artifacts/model" # add here the model uri
aws_account_id = "401605265667"
region = "eu-central-1"
execution_role_arn = "arn:aws:iam::401605265667:role/service-role/AmazonSageMakerServiceCatalogProductsUseRole"
tag_id = '2.8.1'
image_url = "401605265667.dkr.ecr.eu-central-1.amazonaws.com/for-sagemaker-deployment:" + tag_id

# configuration should be revised !
config = {
    "execution_role_arn": execution_role_arn,
    "bucket_name": "xgboostbucket",
    "image_url": image_url,
    "region_name": region,
    "archive": False,
    "instance_type": "ml.m5.large", #  choose a smaller model
    "instance_count": 1,
    "synchronous": True,
    "timeout_seconds": 3600,
    "variant_name": "prod-variant-1",
    "tags": {"training_timestamp": "2023-11-22"},
}

## Deploy to SageMaker

In [ ]:
client = get_deploy_client("sagemaker")

client.create_deployment(
    name="stockmodel",
    model_uri=model_uri,
    flavor="python_function",
    config=config,
)

In [2]:
# "s3://mlflow-sagemaker-eu-central-1-401605265667/stockmodel-model-auv4smcgregdkkkujueyqq/"

from mlflow.deployments import get_deploy_client
#client = get_deploy_client("sagemaker:/eu-central-1/arn:aws:sagemaker:eu-central-1:401605265667:endpoint/stockmodel")
#client = get_deploy_client("sagemaker:/eu-central-1/arn:aws:iam::401605265667:user/mluser")
#client = get_deploy_client("sagemaker:/eu-central-1/arn:aws:iam::401605265667:role/sagemakerrole")
#client = get_deploy_client("sagemaker:/eu-central-1/arn:aws:iam::401605265667:role/service-role/AmazonSageMakerServiceCatalogProductsUseRole")
#client = get_deploy_client(f"sagemaker:/{region}/{model_role}")
#client.predict("stockmodel", df)

In [8]:
df  = pd.read_csv("./data/sp500_predict_2023-11-16.csv")
df.drop("Date",axis=1, inplace=True)
query_input = df.to_dict(orient='split')
data = {"dataframe_split": query_input}
byte_data = json.dumps(data).encode('utf-8')

In [ ]:
model_role= "arn:aws:iam::401605265667:user/mluser"

In [9]:
global app_name
global region
app_name = 'stockmodel'
region = "eu-central-1"

def check_status(app_name):
    sage_client = boto3.client('sagemaker', region_name=region)
    endpoint_description = sage_client.describe_endpoint(EndpointName=app_name)
    endpoint_status = endpoint_description['EndpointStatus']
    return endpoint_status

def query_endpoint(app_name, input_json):
    client = boto3.session.Session().client('sagemaker-runtime', region)
    response = client.invoke_endpoint(
        EndpointName = app_name,
        Body = input_json,
        ContentType = 'application/json'
    )
    preds = response['Body'].read().decode('ascii')
    preds = json.loads(preds)
    print('Received response: {}'.format(preds))
    return preds

In [10]:
query_endpoint(app_name, input_json=byte_data)

Received response: {'predictions': [1]}


{'predictions': [1]}

## Consol Commands for the Deployment

In [ ]:
##     model_uri="/home/erik/DataspellProjects/Stock-Prediction/mlruns/467592723741106657/c413fe91b94f4a8db7492ffa6657a0f6/artifacts/model", 


## systemctl --user start docker-desktop

## Container mit aws

# mlflow sagemaker build-and-push-container --build --push -c for-sagemaker-deployment
# aws sagemaker build-and-push-container

## Container mit Docker 

# docker build -t xgboostimage
# aws ecr get-login-password --region eu-central-1 | docker login --username AWS --password-stdin 401605265667.dkr.ecr.eu-central-1.amazonaws.com
# aws ecr create-repository --repository-name xgboost_repo
# docker tag xgboost_image:latest 401605265667.dkr.ecr.eu-central-1.amazonaws.com/xgboost_repo:latest
# docker push 401605265667.dkr.ecr.eu-central-1.amazonaws.com/xgboost_repo:latest


# mlflow sagemaker deploy --app-name xformer -m runs:/7-16420961991120000000062/model --execution-role-arn arn:aws:iam::612652722220:role/role-for-mlflow-sagemaker-deploy 
# --bucket jaganes-sagemaker --image-url 612652722220.dkr.ecr.us-east-1.amazonaws.com/for-sagemaker-deployment:1.22.0 
# --region-name us-east-1 --mode create 
# --instance-type ml.m5.large --instance-count 1 
# --flavor python_function

## Endpoint Request

In [ ]:
global app_name
global region
app_name = 'my-deployment-logisticRegression'
region = "eu-central-1"

def check_status(app_name):
    sage_client = boto3.client('sagemaker', region_name=region)
    endpoint_description = sage_client.describe_endpoint(EndpointName=app_name)
    endpoint_status = endpoint_description['EndpointStatus']
    return endpoint_status

def query_endpoint(app_name, input_json):
    client = boto3.session.Session().client('sagemaker-runtime', region)
    response = client.invoke_endpoint(
        EndpointName = app_name,
        Body = input_json,
        ContentType = 'application/json'#'; format=pandas-split',
    )
    preds = response['Body'].read().decode('ascii')
    preds = json.loads(preds)
    print('Received response: {}'.format(preds))
    return preds

# Check endpoint status
print('Application status is {}'.format(check_status(app_name)))

arr_predict = np.random.randn(2,3)
query_input = pd.DataFrame(arr_predict).to_dict(orient='split')
print(query_input)

data = {"dataframe_split": query_input}
byte_data = json.dumps(data).encode('utf-8')
predictions = query_endpoint(app_name=app_name, input_json=byte_data)
print(predictions)